In [2]:
from os import environ

input_folder = environ.get(
    'CROSSCOMPUTE_INPUT_FOLDER', 'tests/standard/input')
output_folder = environ.get(
    'CROSSCOMPUTE_OUTPUT_FOLDER', 'tests/standard/output')

In [6]:
import pandas as pd
from os.path import join, splitext

specifications_path = join(output_folder, 'specifications.xlsx')
json_output_path = join(output_folder, 'assets.json')

excel_writer = pd.ExcelWriter(specifications_path)
for file_name in [
    'specs_data.csv',
    'scenario_info.csv',
    'scenario_parameters.csv',
]:
    table = pd.read_csv(join(input_folder, file_name))
    base_name = splitext(file_name)[0]
    sheet_name = ''.join(_.title() for _ in base_name.split('_'))
    table.to_excel(excel_writer, sheet_name=sheet_name, index=False)
excel_writer.save()

In [7]:
from os import makedirs
from os.path import join

details_folder = join(output_folder, 'details')
summaries_folder = join(output_folder, 'summaries')
try:
    makedirs(details_folder)
    makedirs(summaries_folder)
except OSError:
    pass

In [8]:
from onsset.runner import calibration, scenario

cities_path = join(input_folder, 'cities.csv')
calibrated_specifications_path = join(output_folder, 'specifications.xlsx')
calibrated_cities_path = join(output_folder, 'cities.csv') 

calibration(
    specs_path=specifications_path,
    csv_path=cities_path,
    specs_path_calib=calibrated_specifications_path,
    calibrated_csv_path=calibrated_cities_path)
scenario(
    specs_path=calibrated_specifications_path,
    calibrated_csv_path=calibrated_cities_path,
    results_folder=details_folder,
    summary_folder=summaries_folder)

2021-06-16 19:04:28,469		Populate ResidentialDemandTier columns
2021-06-16 19:04:28,473		Ensure that columns that are supposed to be numeric are numeric
2021-06-16 19:04:28,502		NumExpr defaulting to 2 threads.
2021-06-16 19:04:28,505		Replace null values with zero
2021-06-16 19:04:28,506		Sort by country, Y and X
2021-06-16 19:04:28,513		Classify road dist
2021-06-16 19:04:28,518		Classify substation dist
2021-06-16 19:04:28,521		Classify elevation
2021-06-16 19:04:28,524		Classify slope
2021-06-16 19:04:28,527		Classify land cover
2021-06-16 19:04:28,530		Combined classification
2021-06-16 19:04:28,533		Grid penalty
2021-06-16 19:04:28,536		Calculate Wind CF
2021-06-16 19:04:28,568		Population calibration process
2021-06-16 19:04:28,592		Urban/rural calibration process
2021-06-16 19:04:28,706		Population projection process


The calibrated population differs by 0.00. In case this is not acceptable please revise this part of the code


2021-06-16 19:04:28,834		Calibrate current electrification
2021-06-16 19:04:28,925		Calibration finished. Results are transferred to the csv file


We have identified the existence of transformers or MV lines as input data; therefore we proceed using those for the calibration
The modelled electrification rate differ by 0.00. Urban elec. rate differ by 0.00 and Rural elec. rate differ by 0.00. 
If this is not acceptable please revise this part of the algorithm
Djibouti
Scenario: 1


2021-06-16 19:04:29,137		Determine current MV line length
2021-06-16 19:04:29,146		Calculate new connections
2021-06-16 19:04:29,151		Setting electrification demand as per target per year
2021-06-16 19:04:29,187		Calculate minigrid hydro LCOE
2021-06-16 19:04:29,251		Calculate minigrid PV LCOE
2021-06-16 19:04:29,314		Calculate minigrid wind LCOE
2021-06-16 19:04:29,378		Calculate standalone PV LCOE
2021-06-16 19:04:29,410		Determine minimum technology (off-grid)
2021-06-16 19:04:29,436		Determine minimum tech LCOE
2021-06-16 19:04:29,449		Define the initial electrification status
2021-06-16 19:04:29,579		Initially 245 electrified
2021-06-16 19:04:29,723		Electrification loop 1 with 245 electrified
2021-06-16 19:04:29,835		Calculate investment cost
2021-06-16 19:04:29,844		Determine electrification limits
2021-06-16 19:04:29,856		Determine final electrification decision
2021-06-16 19:04:29,858		Calculate new capacity
2021-06-16 19:04:29,874		Calculate summaries
2021-06-16 19:04:29,923	

The electrification rate achieved in 2025 is 83.3 %


2021-06-16 19:04:30,080		Calculate minigrid wind LCOE
2021-06-16 19:04:30,144		Calculate standalone PV LCOE
2021-06-16 19:04:30,177		Determine minimum technology (off-grid)
2021-06-16 19:04:30,201		Determine minimum tech LCOE
2021-06-16 19:04:30,214		Define the initial electrification status
2021-06-16 19:04:30,347		Initially 245 electrified
2021-06-16 19:04:30,488		Electrification loop 1 with 453 electrified
2021-06-16 19:04:30,569		Electrification loop 2 with 49 electrified
2021-06-16 19:04:30,650		Electrification loop 3 with 8 electrified
2021-06-16 19:04:30,731		Electrification loop 4 with 1 electrified
2021-06-16 19:04:30,760		Calculate investment cost
2021-06-16 19:04:30,768		Determine electrification limits
2021-06-16 19:04:30,776		Determine final electrification decision
2021-06-16 19:04:30,778		Calculate new capacity
2021-06-16 19:04:30,794		Calculate summaries


The electrification rate achieved in 2030 is 100.0 %


2021-06-16 19:04:31,021		Finished


In [9]:
from glob import glob
from os.path import basename, splitext

def collect_scenarios(target_path, source_folder):
    tables = []
    for path in glob(join(source_folder, '*.csv')):
        table = pd.read_csv(path)
        table['ScenarioName'] = basename(splitext(path)[0])
        columns = list(table.columns)
        columns.insert(0, columns.pop(columns.index('ScenarioName')))
        tables.append(table[columns])
    scenarios = pd.concat(tables)
    scenarios.to_csv(target_path, index=False)
    return scenarios
    
scenarios = collect_scenarios(join(output_folder, 'details.csv'), details_folder)
summaries = collect_scenarios(join(output_folder, 'summaries.csv'), summaries_folder)

In [10]:
report_year = summaries.columns[-1]
report_year

'2030'

In [24]:
t = summaries[['Unnamed: 0', report_year]]
t = t.set_index('Unnamed: 0')
t = t.squeeze()
t = t[t != 0]
t

Unnamed: 0
1.Population_Grid            1.111083e+06
1.Population_SA_PV           3.546339e+04
1.Population_MG_PV           3.195504e+04
1.Population_MG_Wind         6.488908e+02
2.New_Connections_Grid       2.058052e+05
2.New_Connections_SA_PV      3.546339e+04
2.New_Connections_MG_PV      3.195504e+04
2.New_Connections_MG_Wind    6.488908e+02
3.Capacity_Grid              9.644907e+03
3.Capacity_SA_PV             2.017458e+03
3.Capacity_MG_PV             2.037226e+03
3.Capacity_MG_Wind           2.835530e+01
4.Investment_Grid            3.367748e+07
4.Investment_SA_PV           5.261939e+06
4.Investment_MG_PV           4.632137e+06
4.Investment_MG_Wind         9.178262e+04
Name: 2030, dtype: float64

In [29]:
population_keys = [_ for _ in t.keys() if _.startswith('1.Population')]
population_keys

['1.Population_Grid',
 '1.Population_SA_PV',
 '1.Population_MG_PV',
 '1.Population_MG_Wind']

In [30]:
technology_codes = [_.replace('1.Population_', '') for _ in population_keys]
technology_codes

['Grid', 'SA_PV', 'MG_PV', 'MG_Wind']

In [36]:
technology_name_by_code = {
    'Grid': 'Grid Extension',
    'SA_PV': 'Stand Alone - Photovoltaic',
    'MG_PV': 'Mini Grid - Photovoltaic',
    'MG_Wind': 'Mini Grid - Wind',
}
technology_names = [technology_name_by_code[_] for _ in technology_codes]
technology_names

['Grid Extension',
 'Stand Alone - Photovoltaic',
 'Mini Grid - Photovoltaic',
 'Mini Grid - Wind']

In [37]:
import pandas as pd
summary_table = pd.DataFrame()
summary_table['Technology'] = technology_names

In [39]:
t

Unnamed: 0
1.Population_Grid            1.111083e+06
1.Population_SA_PV           3.546339e+04
1.Population_MG_PV           3.195504e+04
1.Population_MG_Wind         6.488908e+02
2.New_Connections_Grid       2.058052e+05
2.New_Connections_SA_PV      3.546339e+04
2.New_Connections_MG_PV      3.195504e+04
2.New_Connections_MG_Wind    6.488908e+02
3.Capacity_Grid              9.644907e+03
3.Capacity_SA_PV             2.017458e+03
3.Capacity_MG_PV             2.037226e+03
3.Capacity_MG_Wind           2.835530e+01
4.Investment_Grid            3.367748e+07
4.Investment_SA_PV           5.261939e+06
4.Investment_MG_PV           4.632137e+06
4.Investment_MG_Wind         9.178262e+04
Name: 2030, dtype: float64

In [38]:
summary_table['Population Connected']

,Technology
0,Grid Extension
1,Stand Alone - Photovoltaic
2,Mini Grid - Photovoltaic
3,Mini Grid - Wind


In [ ]:
summary_table['Investment Required']

In [ ]:
summary_table['Added Capacity']